In [1]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import PIL
import imageio
import matplotlib.pyplot as plt
import numpy as np
import skimage.transform
import torch
import torch.nn as nn
import torch.utils.data
import torchvision
from torchvision import datasets
from torchvision import transforms
import random
from IPython import display
from torch.utils.data import TensorDataset, DataLoader

In [2]:
np.random.seed(123)

In [3]:
# The first loader is used for the Face Identification task: a multi-class classification task (hence supervised learning):
from sklearn.datasets import fetch_lfw_people

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
lfw_people = fetch_lfw_people()
#for name in lfw_people.target_names:
 # print(name)

In [5]:
lfw_people.images

array([[[ 34.      ,  29.333334,  22.333334, ...,  20.      ,
          25.666666,  30.666666],
        [ 37.333332,  32.      ,  25.333334, ...,  21.      ,
          26.666666,  32.      ],
        [ 33.333332,  32.333332,  40.333332, ...,  23.666666,
          28.      ,  35.666668],
        ...,
        [166.      ,  97.      ,  44.333332, ...,   9.666667,
          14.333333,  12.333333],
        [ 64.      ,  38.666668,  30.      , ...,  12.666667,
          16.      ,  14.      ],
        [ 30.666666,  29.      ,  26.333334, ...,  14.666667,
          16.      ,  14.      ]],

       [[158.      , 160.66667 , 169.66667 , ...,  74.333336,
          28.      ,  15.666667],
        [156.      , 155.33333 , 163.33333 , ...,  83.      ,
          25.666666,  14.      ],
        [146.66667 , 143.66667 , 144.66667 , ...,  82.333336,
          26.      ,  14.666667],
        ...,
        [118.666664, 120.      , 170.      , ..., 131.33333 ,
         127.333336, 126.      ],
        [125

In [6]:
lfw_people.data.shape

(13233, 2914)

In [7]:
lfw_people.images.shape

(13233, 62, 47)

In [8]:
t_data = torch.from_numpy(lfw_people.images)

In [9]:
my_transforms = transforms.CenterCrop(220)

In [10]:
t_data = my_transforms(t_data)

In [11]:
t_data

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [12]:
t_data.shape

torch.Size([13233, 220, 220])

In [13]:
t_data = t_data.unsqueeze(1)

In [14]:
t_data.shape

torch.Size([13233, 1, 220, 220])

In [15]:
t_data.shape

torch.Size([13233, 1, 220, 220])

In [16]:
stacked_data = np.stack((t_data,)*3, axis=-1)
t_data.shape

torch.Size([13233, 1, 220, 220])

In [11]:
# Each of the 13233 faces is assigned to a single person id in the target array:
lfw_people.target.shape

(13233,)

In [12]:
list(lfw_people.target[:10])

[5360, 3434, 3807, 26, 3957, 439, 2683, 3734, 5465, 3445]

In [13]:
lfw_people.target

array([5360, 3434, 3807, ..., 2175,  373, 2941])

In [46]:
X_train, X_test, Y_train, Y_test = train_test_split(t_data, lfw_people.target, shuffle=False, test_size=.20)

In [47]:
X_train.shape

torch.Size([10586, 1, 220, 220])

In [48]:
train_data = torch.Tensor(X_train) # transform to torch tensor
train_target = torch.Tensor(Y_train)

my_dataset = TensorDataset(train_data,train_target) # create your datset
train_loader = DataLoader(my_dataset) # create your dataloader

In [49]:
test_data = torch.Tensor(X_test) # transform to torch tensor
test_target = torch.Tensor(Y_test)

dataset = TensorDataset(test_data,test_target) # create your datset
test_loader = DataLoader(dataset) # create your dataloader

In [50]:
class DeepConvolutionalNetwork(nn.Module):
  """Convolutional network that can go arbitrary deep."""
  
  def __init__(self):
    super().__init__()

    conv_blocks = []
    
    conv_blocks.append(nn.Sequential(
        
        #conv 1
      nn.Conv2d(3, 64, kernel_size=7, stride=2),
      #pool 1
      nn.MaxPool2d(2),
      nn.ReLU(inplace=True),

      #conv 2a
      nn.Conv2d(64, 64, kernel_size=1, stride=1),
      nn.ReLU(inplace=True),

      #conv 2
      nn.Conv2d(64, 192, kernel_size=3, stride=1),
      #pool 2
      nn.MaxPool2d(2),
      nn.ReLU(inplace=True),
      
      #conv 3a
      nn.Conv2d(192, 192, kernel_size=1, stride=1),
      nn.ReLU(inplace=True),

      #conv 3
      nn.Conv2d(192, 384, kernel_size=3, stride=1),
      #pool 3
      nn.MaxPool2d(2),
      nn.ReLU(inplace=True),

      #conv 4a
      nn.Conv2d(384, 384, kernel_size=1, stride=1),
      nn.ReLU(inplace=True),

      #conv 4
      nn.Conv2d(384, 256, kernel_size=3, stride=1),
      nn.ReLU(inplace=True),

      #conv 5a
      nn.Conv2d(256, 256, kernel_size=1, stride=1),
      nn.ReLU(inplace=True),

      #conv 5
      nn.Conv2d(256, 256, kernel_size=3, stride=1),
      nn.ReLU(inplace=True),

      #conv 6a
      nn.Conv2d(256, 256, kernel_size=1, stride=1),
      nn.ReLU(inplace=True),

      #conv 6
      nn.Conv2d(256, 256, kernel_size=3, stride=1),
      nn.MaxPool2d(2),
      nn.ReLU(inplace=True),

      #fc 1
      nn.Linear(256, 128),

      #fc 2
      nn.Linear(128, 128),

      #fc 7128
      nn.Linear(128, 128),

    ))
    
    # It's important to wrap layers into ModuleList so the PyTorch knows that
    # the self.conv_blocks has parameters to optimize.
    self.conv_blocks = nn.ModuleList(conv_blocks)
      
   
  def apply_conv_block(self, conv_block, x):
    output = conv_block(x)
    return output

  def forward(self, x):
    for block in self.conv_blocks:
      x = self.apply_conv_block(block, x) 
    x = self.linear(x.view(x.size(0), -1))
    return x

In [1]:
# DeepConvolutionalNetwork()

In [18]:
lfw_people.target

array([5360, 3434, 3807, ..., 2175,  373, 2941])

In [19]:
X_train.shape

torch.Size([10586, 220, 220])

In [51]:
t_data.shape

torch.Size([13233, 1, 220, 220])

In [22]:
#X_train = tf.cast(X_train, dtype=tf.int64)

In [23]:
#X_test = tf.cast(X_test, dtype= tf.int64)

In [52]:
def train(model, criterion, data_loader, optimizer, num_epochs):
    """Simple training loop for a PyTorch model.""" 
    
    # Make sure model is in training mode.
    model.train()
    
    # Move model to the device.
    #model.to(device)
    
    # Exponential moving average of the loss.
    ema_loss = None
    
    # Loop over epochs.
    for epoch in range(num_epochs):    
      
    # Loop over data.
      for batch_idx, (data, target) in enumerate(data_loader):
            
          # Forward pass.
          output = model(data)
          loss = criterion(output, target)
          
          # Backward pass.
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          
          # NOTE: It is important to call .item() on the loss before summing.
          if ema_loss is None:
            ema_loss = loss.item()
          else:
            ema_loss += (loss.item() - ema_loss) * 0.01 
          
      # Print out progress the end of epoch.
      print('Train Epoch: {} \tLoss: {:.6f}'.format(
            epoch, ema_loss),
      )
              
              
def test(model, data_loader):
    """Measures the accuracy of a model on a data set."""
    
    # Make sure the model is in evaluation mode.
    model.eval()
    correct = 0

    # We do not need to maintain intermediate activations while testing.
    with torch.no_grad():   
        
        # Loop over test data.
        for data, target in data_loader:
          
            # Forward pass.
            output = model(data)
            
            # Get the label corresponding to the highest predicted probability.
            pred = output.argmax(dim=1, keepdim=True)
            
            # Count number of correct predictions.
            correct += pred.cpu().eq(target.view_as(pred)).sum().item()

    # Print test accuracy.
    percent = 100. * correct / len(data_loader.dataset)
    print(f'Accuracy: {correct}/{len(data_loader.dataset)} ({percent:.0f}%)')
    return percent
   

In [53]:
print(f'Training ...')
conv_model = DeepConvolutionalNetwork()
optimizer = torch.optim.SGD(conv_model.parameters(), lr=1e-1)

train(conv_model, nn.CrossEntropyLoss(), train_loader, optimizer, num_epochs=1)

accuracy = test(conv_model, test_loader)
print('Done.')

Training ...


RuntimeError: ignored

In [8]:
# The second loader is typically used for the face verification task: each sample is a pair of two picture belonging or not to the same person:
from sklearn.datasets import fetch_lfw_pairs

In [9]:
lfw_pairs_train = fetch_lfw_pairs(subset='train')

In [24]:
lfw_pairs_train

{'DESCR': ".. _labeled_faces_in_the_wild_dataset:\n\nThe Labeled Faces in the Wild face recognition dataset\n------------------------------------------------------\n\nThis dataset is a collection of JPEG pictures of famous people collected\nover the internet, all details are available on the official website:\n\n    http://vis-www.cs.umass.edu/lfw/\n\nEach picture is centered on a single face. The typical task is called\nFace Verification: given a pair of two pictures, a binary classifier\nmust predict whether the two images are from the same person.\n\nAn alternative task, Face Recognition or Face Identification is:\ngiven the picture of the face of an unknown person, identify the name\nof the person by referring to a gallery of previously seen pictures of\nidentified persons.\n\nBoth Face Verification and Face Recognition are tasks that are typically\nperformed on the output of a model trained to perform Face Detection. The\nmost popular model for Face Detection is called Viola-Jones

In [10]:
list(lfw_pairs_train.target_names)

['Different persons', 'Same person']

In [11]:
lfw_pairs_train.pairs.shape

(2200, 2, 62, 47)

In [12]:
lfw_pairs_train.data.shape

(2200, 5828)

In [13]:
lfw_pairs_train.target.shape

(2200,)